This file priviately reproduces vbf-filtered DY in UL setting FROM GEN-SIM sample from RERECO

## NOTE:
for whatever reason, the environement we need doesn't work well with jupyter notebook, so please use the bash commands below from a separate terminal (from start to finish)

Install CMSSW (we assume voms-proxy-init -voms cms is done)

In [ ]:
%%bash
voms-proxy-init -voms cms
export SCRAM_ARCH=slc7_amd64_gcc700
source /cvmfs/cms.cern.ch/cmsset_default.sh

cmsrel CMSSW_10_6_17_patch1
cd CMSSW_10_6_17_patch1/src

eval `scram runtime -sh`
scram b -j8
cd ../..


Step 2 Digi

In [ ]:
%%bash

cmsDriver.py  \
    --python_filename HIG-RunIIFall18wmRerecoDIGI_cfg.py \
    --eventcontent RAWSIM \
    --pileup 2018_25ns_UltraLegacy_PoissonOOTPU \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier GEN-SIM-DIGI \
    --fileout file:HIG-RunIIFall18wmRerecoDIGI.root \
    --pileup_input "dbs:/MinBias_TuneCP5_13TeV-pythia8/RunIISummer20UL18SIM-106X_upgrade2018_realistic_v11_L1v1-v2/GEN-SIM" \
    --conditions 106X_upgrade2018_realistic_v11_L1v1 \
    --step DIGI,L1,DIGI2RAW \
    --geometry DB:Extended \
    --filein file:/eos/purdue//store/mc/RunIIFall18wmLHEGS/DYJetsToLL_M-105To160_VBFFilter_TuneCP5_PSweights_13TeV-amcatnloFXFX-pythia8/GEN-SIM/VBFPostMGFilter_102X_upgrade2018_realistic_v11_ext1-v1/100000/00E0BCDB-7BFA-EC49-A3EE-A7CEC0EFD0CD.root \
    --era Run2_2018 \
    --runUnscheduled \
    --no_exec \
    --mc \
    -n 1000 

time nohup cmsRun HIG-RunIIFall18wmRerecoDIGI_cfg.py &> Step2_out.log

Step 2.5 HLT (stop here and redo all the previous steps with -n 1000 bc this step requires a seperate and new env (idk why)

In [ ]:
%%bash

mkdir HLT_step
cd HLT_step/
export SCRAM_ARCH=slc7_amd64_gcc700
source /cvmfs/cms.cern.ch/cmsset_default.sh

cmsrel CMSSW_10_2_16_UL
cd CMSSW_10_2_16_UL/src/

eval `scram runtime -sh`
scram b

cd ../../..


In [ ]:
%%bash

cmsDriver.py \
    --python_filename HIG-RunIIFall18wmRerecoHLT_cfg.py \
    --eventcontent RAWSIM \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier GEN-SIM-RAW \
    --fileout file:HIG-RunIIFall18wmRerecoHLT.root \
    --conditions 102X_upgrade2018_realistic_v15 \
    --customise_commands 'process.source.bypassVersionCheck = cms.untracked.bool(True)' \
    --step HLT:2018v32 \
    --geometry DB:Extended \
    --filein file:HIG-RunIIFall18wmRerecoDIGI.root \
    --era Run2_2018 \
    --no_exec \
    --mc \
    -n 1000

time nohup  cmsRun HIG-RunIIFall18wmRerecoHLT_cfg.py &> Step2p5_out.log

Step 3 AOD (RECO)

In [ ]:
%%bash

cd CMSSW_10_6_17_patch1/src
eval `scram runtime -sh`
scram b -j8
cd ../..

cmsDriver.py \
    --python_filename HIG-RunIIFall18wmRerecoRECO_cfg.py \
    --eventcontent AODSIM \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier AODSIM \
    --fileout file:HIG-RunIIFall18wmRerecoRECO.root \
    --conditions 106X_upgrade2018_realistic_v11_L1v1 \
    --step RAW2DIGI,L1Reco,RECO,RECOSIM,EI \
    --geometry DB:Extended \
    --filein file:HIG-RunIIFall18wmRerecoHLT.root \
    --era Run2_2018 \
    --runUnscheduled \
    --no_exec \
    --mc \
    --nThreads 4 \
    -n 1000

time nohup  cmsRun HIG-RunIIFall18wmRerecoRECO_cfg.py &> Step3_out.log

Step 4 Mini AOD

In [ ]:
%%bash

cmsDriver.py \
    --python_filename HIG-RunIIFall18wmRerecoMINI_cfg.py \
    --eventcontent MINIAODSIM \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier MINIAODSIM \
    --fileout file:HIG-RunIIFall18wmRerecoMINI.root \
    --conditions 106X_upgrade2018_realistic_v16_L1v1 \
    --step PAT \
    --procModifiers run2_miniAOD_UL \
    --geometry DB:Extended \
    --filein file:HIG-RunIIFall18wmRerecoRECO.root \
    --era Run2_2018 \
    --runUnscheduled \
    --no_exec \
    --mc \
    --nThreads 4 \
    -n 1000

time nohup cmsRun HIG-RunIIFall18wmRerecoMINI_cfg.py &> Step4_out.log

Step 5 Nano AOD

In [ ]:
%%bash


mkdir nano_step
cd nano_step
source /cvmfs/cms.cern.ch/cmsset_default.sh
export SCRAM_ARCH=slc7_amd64_gcc700

cmsrel CMSSW_10_6_26
cd CMSSW_10_6_26/src/
eval `scram runtime -sh`
scram b
cd ../../..


In [ ]:
%%bash

cmsDriver.py \
    --python_filename HIG-RunIIFall18wmRerecoNANO_cfg.py \
    --eventcontent NANOAODSIM \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier NANOAODSIM \
    --fileout file:HIG-RunIIFall18wmRerecoNANO.root\
    --conditions 102X_upgrade2018_realistic_v15 \
    --customise_commands 'process.source.bypassVersionCheck = cms.untracked.bool(True)' \
    --step NANO \
    --nThreads 4 \
    --filein file:HIG-RunIIFall18wmRerecoMINI.root \
    --era Run2_2018,run2_nanoAOD_102Xv1 \
    --no_exec \
    --mc \
    -n 1000

time nohup cmsRun HIG-RunIIFall18wmRerecoNANO_cfg.py &> Step5_out.log

Step 5 extra: Nano AOD v12
source: https://gitlab.cern.ch/cms-nanoAOD/nanoaod-doc/-/wikis/Releases/NanoAODv12

In [ ]:
# this cell below doesn't get nanoAOD v12
%%bash
mkdir nanov12_setup
cd nanov12_setup
export SCRAM_ARCH=el8_amd64_gcc11
source /cvmfs/cms.cern.ch/cmsset_default.sh
cmsrel CMSSW_13_0_13
cd CMSSW_13_0_13/src
eval `scram runtime -sh`
scram b
cd ../..

cmsDriver.py \
    --python_filename HIG-RunIIFall18wmRerecoNANO_cfg.py \
    --eventcontent NANOAODSIM \
    --customise Configuration/DataProcessing/Utils.addMonitoring \
    --datatier NANOAODSIM \
    --fileout file:HIG-RunIIFall18wmRerecoNANO.root\
    --conditions auto:phase1_2018_realistic \
    --step NANO \
    --nThreads 2 \
    --filein file:../HIG-RunIIFall18wmRerecoMINI.root \
    --era Run2_2018,run2_nanoAOD_106Xv2\
    --no_exec \
    --mc \
    -n 10

cmsRun HIG-RunIIFall18wmRerecoNANO_cfg.py